In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import ast
import csv
import collections
from dython import examples, nominal
from itertools import chain, cycle
import krippendorff_alpha as k
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle 
import numpy as np
import random
import researchpy as rp
import scipy.stats as ss
import spacy
from sklearn import preprocessing
import re


from sklearn.feature_extraction.text import CountVectorizer

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 35
pd.options.display.max_colwidth = 800

np.random.seed(1234)
random.seed(1234)

from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times'],'size':12})
#rcParams['font.size'] = 12
#rc('text', usetex=True)

ModuleNotFoundError: No module named 'dython'

In [ ]:
# read aggregated file
gold_file = '../../data/gold/gold_full_data/with_features_annotated_questions_responses_gold.tsv'
gold_df = pd.read_csv(gold_file, index_col=False, sep='\t')

In [ ]:
# convert string columns back to tuples
columns = ['gold_label_set', 'gold_act_set', 'gold_acts','gold_intents','gold_intents_binary', 'gold_intent_set','gold_coarse_labels', 'gold_fine_labels', 
           'gold_workers', 'gold_sentiments', 'gold_explanations',
           'gold_q_intents', 'gold_q_sentiments',
           'gold_work_times', 'gold_assignment_ids', 'gold_hit_ids', 'coarse_labels', 'fine_labels',
           'workers', 'sentiments', 'explanations']
for column in columns:
    gold_df[column] = [ast.literal_eval(x) for x in gold_df[column]]

In [ ]:
# read per annotator file
expanded_file = '../../data/gold/gold_full_data/expanded_with_features_annotated_questions_responses_gold.csv'
expanded_df = pd.read_csv(expanded_file, index_col=False)

In [ ]:
def plot_freqs_counts_dict(counts_dict, plot_title, label_order, colors, y_lim=1.0, hide_x_labels=False):
    fig, ax = plt.subplots()
    subplot_freqs_counts(counts_dict, ax, plot_title, label_order, colors, y_lim, hide_x_labels)
    plt.show()
    
def plot_freqs_counts_overall(df, column, label_order, colors, y_lim=0.8, hide_x_labels=True):
    counts_dict = collections.Counter(list(chain.from_iterable(df[column].values)))
    fig, ax = plt.subplots()
    subplot_freqs_counts(counts_dict, ax, 'All Hearings', label_order, colors, y_lim, hide_x_labels)
    plt.show()
    
def plot_freqs_counts(df, column, label_order, colors, y_lim=0.8, hide_x_labels=True):
    n_hearings = len(df['hearing_id'].unique())
    n_cols=4
    n_rows=math.ceil(n_hearings / n_cols)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
    grouped = df.groupby(['hearing_id'])
    row_idx, col_idx = 0, 0
    n_figs = 0
    for name, group in grouped:
        counts_dict = collections.Counter(list(chain.from_iterable(group[column].values)))
        n_figs+=1
        plot_title = name[-5:]+':'+group['hearing_title'].unique()[0][:20]
        subplot_freqs_counts(counts_dict, axes[row_idx, col_idx], plot_title, label_order, colors, y_lim, hide_x_labels)
        # increment counters
        col_idx += 1
        if col_idx >= n_cols:
            col_idx = 0
            row_idx += 1
    fig.tight_layout()
    plt.show()
    
def subplot_freqs_counts(counts_dict, ax, plot_title, label_order, colors, y_lim, hide_x_labels):
    label_counts_ordered = {}
    for label in label_order:
        if label in counts_dict:
            label_counts_ordered[label]=counts_dict[label]
        else:
            label_counts_ordered[label]=0
    freqs = [val/sum(counts_dict.values()) for val in label_counts_ordered.values()]
    print(plot_title)
    print(freqs)
    ax.set_ylim(0, y_lim)
    if hide_x_labels:
        ax.set_xticklabels([])
    else:
        plt.setp(ax.get_xticklabels(), rotation=90)
    ax.set_title(plot_title)
    rects=ax.bar(label_counts_ordered.keys(), freqs, color=colors)
    ax.set_ylabel('Frequency')
    for rect, label in zip(rects, label_counts_ordered.values()):
        height = rect.get_height()
        if label != 0:
            ax.text(rect.get_x() + rect.get_width() / 2, 1.05*height, 
                        label, 
                        ha='center', va='bottom')
            
def subplot_freqs_counts_stacked(counts_dict, ax, plot_title, label_order, colors, y_lim, hide_x_labels):
    label_counts_ordered = {}
    for label in label_order:
        if label in counts_dict:
            label_counts_ordered[label]=counts_dict[label]
        else:
            label_counts_ordered[label]=0
    freqs = [val/sum(counts_dict.values()) for val in label_counts_ordered.values()]
    
    ax.set_ylim(0, y_lim)
    if hide_x_labels:
        ax.set_xticklabels([])
    else:
        plt.setp(ax.get_xticklabels(), rotation=90)
    ax.set_title(plot_title)
    label_names = list(label_counts_ordered.keys())
    rect1 = ax.bar(label_names[0], freqs[0], color=colors[0], width=0.5)
    rect2 = ax.bar(label_names[1], freqs[1], color=colors[1], width=0.5)
    rect3 = ax.bar(label_names[1], freqs[2], color=colors[2], bottom=freqs[1], width=0.5)
    ax.set_ylabel('Frequency')
    rects = ax.patches
    label_values = list(label_counts_ordered.values())
    
    height1 = rects[0].get_height()
    height2 = rects[1].get_height()
    height3 = rects[2].get_height()
    ax.text(rects[0].get_x() + rects[0].get_width() / 2, 1.05*height1, 
                        label_values[0], 
                        ha='center', va='bottom')
    ax.text(rects[2].get_x() + rects[2].get_width() / 2, 1.05*(height2+height3), 
                        label_values[1]+label_values[2], 
                        ha='center', va='bottom')
    
    ax.text(rects[1].get_x()-0.07, rects[1].get_y(), 
                        label_values[1], 
                        ha='center', va='bottom')
    ax.text(rects[2].get_x()-0.07, rects[2].get_y(),
                        label_values[2], 
                        ha='center', va='bottom')
    
    
    height2 = rects[0].get_height()

# For dataframes
def plot_freqs_counts_df_overall(df, column, label_order, colors, y_lim=0.65, hide_x_labels=True):
    fig, ax = plt.subplots()
    subplot_freqs_counts_df(ax, df, column, 'All Hearings', label_order, colors, y_lim, hide_x_labels)
    plt.show()
    
def plot_freqs_counts_df(df, column, label_order, colors, y_lim=0.65, hide_x_labels=True):
    n_hearings = len(df['hearing_id'].unique())
    n_cols=4
    n_rows=math.ceil(n_hearings / n_cols)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
    grouped = df.groupby(['hearing_id'])
    row_idx, col_idx = 0, 0
    n_figs = 0
    for name, group in grouped:
        n_figs+=1
        plot_title = name[-5:]+':'+group['hearing_title'].unique()[0][:20]
        subplot_freqs_counts_df(axes[row_idx, col_idx], group, column, plot_title, label_order, colors, y_lim, hide_x_labels)
        # increment counters
        col_idx += 1
        if col_idx >= n_cols:
            col_idx = 0
            row_idx += 1
    fig.tight_layout()
    plt.show()
    
def subplot_freqs_counts_df(ax, df, column, plot_title, label_order, colors, y_lim, hide_x_labels):
    label_set_counts_ordered = {}
    label_set_counts = df[column].value_counts()
    for label in label_order:
        if label in df[column].value_counts():
            label_set_counts_ordered[str(label)]=label_set_counts[label]
        else:
            label_set_counts_ordered[str(label)]=0
    freqs = [val/len(df) for val in label_set_counts_ordered.values()]
    ax.set_ylim(0, y_lim)
    if hide_x_labels:
        ax.set_xticklabels([])
    else:
        plt.setp(ax.get_xticklabels(), rotation=90)
    ax.set_title(plot_title)
    rects=ax.bar(label_set_counts_ordered.keys(), freqs, color=colors)
    ax.set_ylabel('Frequency')
    for rect, label in zip(rects, label_set_counts_ordered.values()):
        height = rect.get_height()
        if label != 0:
            ax.text(rect.get_x() + rect.get_width() / 2, 1.05*height, 
                        label, 
                        ha='center', va='bottom')

def plot_disagreement_time(df, column):
    n_hearings = len(df['hearing_id'].unique())
    n_cols=4
    n_rows=math.ceil(n_hearings / n_cols)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
    grouped = df.groupby(['hearing_id'])
    row_idx, col_idx = 0, 0
    n_figs = 0
    for name, group in grouped:
        n_figs+=1
        plot_title = name[-5:]+':'+group['hearing_title'].unique()[0][:20]
        subplot_disagreement_time(axes[row_idx, col_idx], group, column, plot_title)
        # increment counters
        col_idx += 1
        if col_idx >= n_cols:
            col_idx = 0
            row_idx += 1
    fig.tight_layout()
    plt.show()
    
def subplot_disagreement_time(ax, df, column, plot_title):
    ax.set_yticklabels(range(0,4))
    ax.set_title(plot_title)
    ax.set_ylabel('Number of labels')
    y = [len(labels) for labels in df[column].values]
    x = range(1,51)
    ax.plot(x,y,marker='o', linestyle=':')

## Q0: What is the overall label set distribution?

In [ ]:
colors = plt.get_cmap('tab10').colors

#label order
label_order = ['answer', 'answer_overans-sway', 'shift-dodge', 'shift-correct', 'cant-answer-lying', 'cant-answer-sincere']
# label colors
color_iter = cycle(colors)
label_colors = []
for i in range(len(label_order)):
    label_colors.append(next(color_iter))
    
#label set order
label_set_order = gold_df['gold_label_set'].value_counts(sort=True).index
#label set colors
color_iter = cycle(colors)
label_set_colors = []
for i in range(len(label_set_order)):
    label_set_colors.append(next(color_iter))

### Individual labels

In [ ]:
plot_freqs_counts_overall(gold_df, 'gold_coarse_labels', label_order, label_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts(gold_df, 'gold_coarse_labels', label_order, label_colors, hide_x_labels=True)

### Label sets

In [ ]:
plot_freqs_counts_df_overall(gold_df, 'gold_label_set', label_set_order, label_set_colors, hide_x_labels=False)

In [ ]:
counts_dict = collections.Counter(list(chain.from_iterable(gold_df['gold_label_set'].values)))


In [ ]:
plot_freqs_counts_df(gold_df, 'gold_label_set', label_set_order, label_set_colors)

## Conversational acts

In [ ]:
expanded_df['gold_act'].value_counts().plot(kind='bar')

In [ ]:
expanded_df['gold_intent'].value_counts().plot(kind='bar')

## Q1: Is there disagreement?

In [ ]:
unambiguous_df = gold_df[gold_df['gold_label_set'].str.len()==1]
disagree_df = gold_df[gold_df['gold_label_set'].str.len()>1]
two_disagree_df = gold_df[gold_df['gold_label_set'].str.len()==2]
three_disagree_df = gold_df[gold_df['gold_label_set'].str.len()==3]

label_set_disagree_order = disagree_df['gold_label_set'].value_counts(sort=True).index

In [ ]:
modes_dict = {'Unambiguous': len(unambiguous_df), '2-way disagreement': len(two_disagree_df),
              '3-way disagreement': len(three_disagree_df)}
plot_freqs_counts_dict(modes_dict, 'Response Label Disagreements', modes_dict.keys(), label_colors)

In [ ]:
y_lim=1.0
hide_x_labels=False
column='gold_label_set'
n_hearings = len(gold_df['hearing_id'].unique())
n_cols=4
n_rows=math.ceil(n_hearings / n_cols)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
grouped = gold_df.groupby(['hearing_id'])
row_idx, col_idx = 0, 0
for name, group in grouped:
    plot_title = name[-5:]+':'+group['hearing_title'].unique()[0][:20]
    g_unambiguous_df = group[group[column].str.len()==1]
    g_two_disagree_df = group[group[column].str.len()==2]
    g_three_disagree_df = group[group[column].str.len()==3]
    g_modes_dict = {'Unambiguous': len(g_unambiguous_df), '2-way': len(g_two_disagree_df),
                   '3-way': len(g_three_disagree_df)}
    subplot_freqs_counts_stacked(g_modes_dict, axes[row_idx, col_idx], plot_title, g_modes_dict.keys(), label_colors, y_lim, hide_x_labels=True)
    # increment counters
    col_idx += 1
    if col_idx >= n_cols:
        col_idx = 0
        row_idx += 1
fig.tight_layout()
plt.show()
    

### Unambiguous: No disagreement

In [ ]:
plot_freqs_counts_overall(unambiguous_df, 'gold_label_set', label_order, label_colors, y_lim=0.95, hide_x_labels=False)

In [ ]:
plot_freqs_counts(unambiguous_df, 'gold_label_set', label_order, label_colors, y_lim=0.95)

### Any disagreement (>1)

In [ ]:
plot_freqs_counts_df_overall(disagree_df, 'gold_label_set', label_set_disagree_order, label_set_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts_overall(disagree_df, 'gold_label_set', label_order, label_colors, y_lim=0.95, hide_x_labels=False)

In [ ]:
plot_freqs_counts(disagree_df, 'gold_label_set', label_order, label_colors, y_lim=0.95)

In [ ]:
plot_freqs_counts_df_overall(disagree_df, 'gold_label_set', label_set_order, label_set_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts_df_overall(two_disagree_df, 'gold_label_set', label_set_order, label_set_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts_df(disagree_df, 'gold_label_set', label_set_order, label_set_colors)

### Two way: Some disagreement

In [ ]:
plot_freqs_counts_overall(two_disagree_df, 'gold_label_set', label_order, label_colors, y_lim=0.95, hide_x_labels=False)

In [ ]:
plot_freqs_counts(two_disagree_df, 'gold_label_set', label_order, label_colors, y_lim=0.95)

In [ ]:
plot_freqs_counts_df_overall(two_disagree_df, 'gold_label_set', label_set_order, label_set_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts_df(two_disagree_df, 'gold_label_set', label_set_order, label_set_colors)

### Three way: Lots of disagreement

In [ ]:
plot_freqs_counts_overall(three_disagree_df, 'gold_label_set', label_order, label_colors, y_lim=0.95, hide_x_labels=False)

In [ ]:
plot_freqs_counts(three_disagree_df, 'gold_label_set', label_order, label_colors, y_lim=0.95)

In [ ]:
plot_freqs_counts_df_overall(three_disagree_df, 'gold_label_set', label_set_order, label_set_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts_df(three_disagree_df, 'gold_label_set', label_set_order, label_set_colors, y_lim=1)

### Disagreement over time

In [ ]:
plot_disagreement_time(gold_df, 'gold_label_set')

In [ ]:
from matplotlib.ticker import AutoMinorLocator
def subplot_scatter_sentiments(df, ax, plot_title):
    one_sided = []
    q_sentiments = df.groupby(['qa_index'])['gold_sentiment_num'].value_counts()
    y_labels = q_sentiments.index.levels[0]
    xs, ys, sizes, colors = [], [], [], []
    for y_idx, y_label in enumerate(y_labels):
        new_x = q_sentiments[y_label].index.values
        if np.all(new_x > 0) or np.all(new_x < 0) or np.all(new_x == 0):
            colors.extend(['r']*len(new_x))
            one_sided.append([new_x,q_sentiments[y_label].values])
        else:
            colors.extend(['b']*len(new_x))
        xs.extend(new_x)
        sizes.extend(q_sentiments[y_label].values*10)
        ys.extend([y_idx]*len(new_x))
    ax.set_title(plot_title)
    ax.scatter(xs,ys,s=sizes,alpha=0.3,color=colors)
    ax.yaxis.grid(True, linewidth=0.4, which='both',markevery=1)
    ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    print(df['hearing_id'].unique())
    print('One sided', len(one_sided), one_sided)

In [ ]:
column='gold_sentiment'
n_hearings = len(expanded_df['hearing_id'].unique())
n_cols=4
n_rows=math.ceil(n_hearings / n_cols)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
grouped_h = expanded_df.groupby(['hearing_id'])
row_idx, col_idx = 0, 0
for name, group_h in grouped_h:
    plot_title = name[-5:]+':'+group_h['hearing_title'].unique()[0][:20]
    subplot_scatter_sentiments(group_h, axes[row_idx, col_idx], plot_title)
    
    # increment counters
    col_idx += 1
    if col_idx >= n_cols:
        col_idx = 0
        row_idx += 1
fig.tight_layout()
plt.show()
    

In [ ]:
sent_order=['veryNegative','negative','somewhatNegative','neutral',
            'somewhatPositive', 'positive', 'veryPositive']
plot_freqs_counts_overall(gold_df, 'gold_sentiments', sent_order, label_colors, hide_x_labels=False)

In [ ]:
plot_freqs_counts(gold_df, 'gold_sentiments', sent_order, label_colors, y_lim=0.95)

## Q2: Is disagreement 'real'?

### 1. IAA

In [ ]:
def calculate_iaa(df, intent_label, intent_code_dict):
    intent_agreement = pd.DataFrame(columns=df['qa_index'].unique(),index=list(set(chain.from_iterable(df['gold_workers'].values))))
    for item in intent_agreement.columns:
        labels, workers = df[df['qa_index']==item][[intent_label,'gold_workers']].values[0]
        for label, worker in zip(labels, workers):
            intent_agreement.loc[worker,item] = intent_code_dict[label]
    print(" #workers, #items:",intent_agreement.shape)
    intent_agreement = intent_agreement.fillna('*')
    intent_agreement_array = intent_agreement.to_numpy()
    missing = "*"
    return(k.krippendorff_alpha(intent_agreement_array, k.nominal_metric, missing_items=missing))
    
def calculate_iaa_sentiment(df, intent_label, intent_code_dict, sentiment_labels):
    missing = "*"
    for sentiment_label in sentiment_labels:
        mask = df['gold_sentiments'].apply(lambda x: sentiment_label in x)
        df_sentiment = df[mask]
        intent_agreement = pd.DataFrame(columns=df_sentiment['qa_index'].unique())
        for item in intent_agreement.columns:
            sentiments, workers, intents = df_sentiment[df_sentiment['qa_index']==item][['gold_sentiments','gold_workers',intent_label]].values[0]
            for sentiment, worker, intent in zip(sentiments, workers, intents):
                if sentiment==sentiment_label:
                    intent_agreement.loc[worker, item] = intent_code_dict[intent]
        intent_agreement.dropna(how='all', axis=0, inplace=True)
        intent_agreement = intent_agreement.fillna('*')
        intent_agreement_array = intent_agreement.to_numpy()
        print("Processing sentiment ", sentiment_label," #workers, #items:",intent_agreement.shape)
        try:
            print(k.krippendorff_alpha(intent_agreement_array, k.nominal_metric, missing_items=missing))
        except ValueError as e1:
            print("Caught ValueError exception: ", e1)
        except Exception as e2:
            print("Caught other exception: ", e2)
            
def calculate_iaa_sentiment_coarse(df, intent_label, intent_code_dict):
    missing = "*"
    sentiment_coarse_to_fine = {'negative':['veryNegative','negative','somewhatNegative'],
                                'neutral':['neutral'],
                                'positive':['somewhatPositive', 'positive', 'veryPositive']}
    for sentiment_group, sentiment_labels in sentiment_coarse_to_fine.items():
        mask = df['gold_sentiments'].apply(lambda x: any(item for item in sentiment_labels if item in x))
        df_sentiment = df[mask]
        intent_agreement = pd.DataFrame(columns=df_sentiment['qa_index'].unique())
        for item in intent_agreement.columns:
            sentiments, workers, intents = df_sentiment[df_sentiment['qa_index']==item][['gold_sentiments','gold_workers',intent_label]].values[0]
            for sentiment, worker, intent in zip(sentiments, workers, intents):
                if sentiment in sentiment_labels:
                    intent_agreement.loc[worker, item] = intent_code_dict[intent]
        intent_agreement.dropna(how='all', axis=0, inplace=True)
        intent_agreement = intent_agreement.fillna('*')
        intent_agreement_array = intent_agreement.to_numpy()
        print("Processing sentiment group", sentiment_group," #workers, #items:",intent_agreement.shape)
        try:
            print(k.krippendorff_alpha(intent_agreement_array, k.nominal_metric, missing_items=missing))
        except ValueError as e1:
            print("Caught ValueError exception: ", e1)
        except Exception as e2:
            print("Caught other exception: ", e2)

#### Overall

In [ ]:
act_intent_to_code = {'answer':0, 'answer_overans-sway':1, 
                 'shift-dodge':2, 'shift-correct':3,
                 'cant-answer-lying':4, 'cant-answer-sincere':5}
k_all = calculate_iaa(gold_df, 'gold_coarse_labels', act_intent_to_code)
print(k_all)

In [ ]:
act_to_code = {'answer':0, 'shift':1, 'cant-answer':2}
k_act_all = calculate_iaa(gold_df, 'gold_acts', act_to_code)
print(k_act_all)

In [ ]:
intent_to_code = {'direct':0, 'overans':1, 'dodge':2, 'correct':3, 
                  'lying':4, 'sincere':5}
k_intent_all = calculate_iaa(gold_df, 'gold_intents', intent_to_code)
print(k_intent_all)

In [ ]:
intent_binary_to_code = {0:0, 1:1}
k_intent_binary_all = calculate_iaa(gold_df, 'gold_intents_binary', intent_binary_to_code)
print(k_intent_binary_all)

In [ ]:
k_unambiguous = calculate_iaa(unambiguous_df, 'gold_coarse_labels', act_intent_to_code)
print(k_unambiguous)

In [ ]:
k_disagree = calculate_iaa(disagree_df, 'gold_coarse_labels', act_intent_to_code)
print(k_disagree)

In [ ]:
k_act_disagree = calculate_iaa(disagree_df, 'gold_acts', act_to_code)
print(k_act_all)

In [ ]:
k_intent_disagree = calculate_iaa(disagree_df, 'gold_intents', intent_to_code)
print(k_intent_disagree)

In [ ]:
k_intent_binary_disagree = calculate_iaa(disagree_df, 'gold_intents_binary', intent_binary_to_code)
print(k_intent_binary_disagree)

In [ ]:
k_two_disagree = calculate_iaa(two_disagree_df, 'gold_coarse_labels', act_intent_to_code)
print(k_two_disagree)

In [ ]:
k_three_disagree = calculate_iaa(three_disagree_df, 'gold_coarse_labels', act_intent_to_code)
print(k_three_disagree)

#### By sentiment: fine-grained

In [ ]:
sentiment_labels = ['veryNegative', 'negative', 'somewhatNegative',
                     'neutral',
                     'somewhatPositive', 'positive', 'veryPositive']

In [ ]:
calculate_iaa_sentiment(gold_df, 'gold_coarse_labels', act_intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(disagree_df, 'gold_coarse_labels', act_intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(two_disagree_df, 'gold_coarse_labels', act_intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(three_disagree_df, 'gold_coarse_labels', act_intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(gold_df, 'gold_acts', act_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(disagree_df, 'gold_acts', act_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(two_disagree_df, 'gold_acts', act_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(three_disagree_df, 'gold_acts', act_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(gold_df, 'gold_intents', intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(disagree_df, 'gold_intents', intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(two_disagree_df, 'gold_intents', intent_to_code, sentiment_labels)

In [ ]:
calculate_iaa_sentiment(three_disagree_df, 'gold_intents', intent_to_code, sentiment_labels)

#### By sentiment: coarse-grained

In [ ]:
calculate_iaa_sentiment_coarse(gold_df, 'gold_coarse_labels', act_intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(disagree_df, 'gold_coarse_labels', act_intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(two_disagree_df, 'gold_coarse_labels', act_intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(three_disagree_df, 'gold_coarse_labels', act_intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(gold_df, 'gold_acts', act_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(disagree_df, 'gold_acts', act_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(two_disagree_df, 'gold_acts', act_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(three_disagree_df, 'gold_acts', act_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(gold_df, 'gold_intents', intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(disagree_df, 'gold_intents', intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(two_disagree_df, 'gold_intents', intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(three_disagree_df, 'gold_intents', intent_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(gold_df, 'gold_intents_binary', intent_binary_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(disagree_df, 'gold_intents_binary', intent_binary_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(two_disagree_df, 'gold_intents_binary', intent_binary_to_code)

In [ ]:
calculate_iaa_sentiment_coarse(three_disagree_df, 'gold_intents_binary', intent_binary_to_code)

#### Reference IAA for perfect disagreement

In [ ]:
num_items = 533 #533, 448(446), 85 (84)
num_annotators=62
num_annotators_per_hit = 6
modes_to_items = {2: (446,[(158,[0,2]), 
                            (82,[2,3]),
                            (56,[0,1]),
                            (50,[4,5]), 
                            (40,[0,3]),
                            (16,[2,5]),
                            (14,[0,5]),
                            (12,[3,5]),
                            (10,[2,5]),
                            (8, [1,2])]),
                 3: (87, [(36, [0,2,3]),
                          (18, [0,1,3]),
                          (9,  [2,3,5]),
                          (6,  [2,4,5]),
                          (6,  [2,3,4]),
                          (6,  [0,4,5]),
                          (3,  [0,1,5]),
                          (3,  [3,4,5])])}
def create_data_frame(num_items, num_annotators, 
                      num_annotators_per_hit, modes_to_items):
    num_annos = num_items * num_annotators_per_hit
    annotator_pattern = list(range(num_annotators))*num_annotators_per_hit
    num_repeats = math.ceil(num_annos/len(annotator_pattern))
    annotators = (num_repeats * annotator_pattern)[:num_annos]
    items = []
    labels = []
    item_id = 0
    for num_modes, (num_items_in_mode, class_proportions) in modes_to_items.items():
        print("Processing mode", num_modes)
        #for item in item_ids[items_added:items_added+num_items_in_mode]: #30
        #print("Processing item id", item)
        print(num_modes)
        print(class_proportions)
        votes_per_class = num_annotators_per_hit//num_modes #6
        for (num_items_in_class, classes) in class_proportions:
            item_votes = []
            for _class in classes:
                item_votes.extend([_class]*votes_per_class)
            print("Votes for item: ", item_votes, ", num items: ", num_items_in_class)
            labels.extend(item_votes*num_items_in_class)
            items.extend(np.repeat((range(item_id,num_items_in_class+item_id)),num_annotators_per_hit))
            item_id += num_items_in_class
        #items_added += num_items_in_mode
    print(len(labels), labels)    
    print(len(items), items)
    print(len(annotators), annotators)

    # sanity check
    expected_length = num_items * num_annotators_per_hit
    assert(len(items) == expected_length)
    assert(len(annotators) == expected_length)
    assert(len(labels) == expected_length)

    # create dataframe
    data = zip(items, annotators, labels)
    df = pd.DataFrame(list(data), columns = ['qa_index', 'worker_id', 'intent'])
    return df

In [ ]:
fake_df = create_data_frame(num_items, num_annotators, num_annotators_per_hit, modes_to_items)

In [ ]:
def calculate_iaa_simple(df, coded_label):
    grouped = df.sort_values(by=['worker_id']).groupby(['worker_id'])
    anno_to_items = {}
    for name, group in grouped:
        anno_to_items[name] = group['qa_index'].unique()
    intent_agreement = pd.DataFrame(columns=df['qa_index'].unique(), index=df['worker_id'].unique())
    annotators = df['worker_id'].unique()
    for annotator in annotators:
        for item in anno_to_items[annotator]:
            intent_agreement.loc[annotator,item] = df.loc[(df['worker_id']==annotator) & (df['qa_index']==item), [coded_label]].to_numpy()[0][0]
    intent_agreement = intent_agreement.fillna("*")
    intent_agreement_array = intent_agreement.to_numpy()
    missing = "*"
    return(k.krippendorff_alpha(intent_agreement_array, k.nominal_metric, missing_items=missing))

In [ ]:
k_fake = calculate_iaa_simple(fake_df, 'intent')
print(k_fake)

# Q3: Is intent correlated with other factors?

In [ ]:
def get_correlations(df, nominal_columns):
    print("Cramer's V")
    fig, ax = plt.subplots(figsize=(14, 10))
    print(nominal.associations(df, theil_u=False, plot=True, return_results=True, ax=ax, nominal_columns=nominal_columns))
    fig.tight_layout()
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.show()
    print("Theil's U")
    fig, ax = plt.subplots(figsize=(14, 10))
    print(nominal.associations(df, theil_u=True, plot=True, return_results=True, ax=ax, nominal_columns=nominal_columns))
    fig.tight_layout()
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.show()

In [ ]:
expanded_df['hit_order'] = pd.to_numeric(expanded_df['hit_order'], downcast='integer')
gold_df['hit_order'] = pd.to_numeric(gold_df['hit_order'], downcast='integer')

In [ ]:
nominal_columns = ['gold_coarse_label', 'gold_act', 'gold_intent', 
       'gold_worker', 
       'r_feature_politeness_==Factuality==',
       'r_feature_politeness_==Hedges==', 'e_feature_politeness_==Please==',
       'l_feature_politeness_==Please_start==',
       'e_feature_politeness_==SUBJUNCTIVE==', 'l_question_lexicon',
       'l_feature_politeness_==Direct_start==', 'q_has_cardinal_num',
                              'e_feature_politeness_==2nd_person==',
       'l_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person==',
       'l_feature_politeness_==HASNEGATIVE==',
       'q_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==Hedges==', 'e_has_cardinal_num', 'r_has_adverb',
       'e_feature_politeness_==Deference==',
                              'l_has_modal',
       'l_feature_politeness_==1st_person==',
       'r_feature_politeness_==SUBJUNCTIVE==',
       'e_feature_politeness_==Apologizing==',
       'q_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==Indirect_(btw)==',
       'e_feature_politeness_==Direct_question==',
       'l_feature_politeness_==1st_person_start==',
       'e_feature_politeness_==2nd_person_start==',
                              'e_feature_politeness_==Factuality==',
       'l_feature_politeness_==SUBJUNCTIVE==',
       'q_feature_politeness_==SUBJUNCTIVE==',
       'e_feature_politeness_==INDICATIVE==',
       'l_feature_politeness_==HASPOSITIVE==',
       'q_feature_politeness_==Indirect_(btw)==',
       'q_feature_politeness_==Indirect_(greeting)==',
       'r_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==HASPOSITIVE==',
                              'l_feature_politeness_==Indirect_(greeting)==', 'l_has_adjective',
       'q_has_adjective', 'e_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==HASHEDGE==',
       'l_feature_politeness_==1st_person_pl.==',
       'r_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==Please_start==',
       'l_feature_politeness_==2nd_person_start==',
                             'r_feature_politeness_==2nd_person==', 'r_has_adjective',
       'l_has_cardinal_num', 'q_feature_politeness_==Factuality==',
       'q_has_named_entity', 'r_feature_politeness_==Gratitude==',
       'r_question_lexicon', 'r_feature_politeness_==Please==',
       'r_feature_politeness_==Indirect_(greeting)==',
                             'q_feature_politeness_==Please_start==', 'q_has_adverb',
       'q_feature_politeness_==1st_person==',
       'r_feature_politeness_==Deference==', 'r_has_modal',
       'r_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Gratitude==',
       'l_feature_politeness_==Indirect_(btw)==',
       'l_feature_politeness_==Hedges==',
                              'q_feature_politeness_==Please==',
       'l_has_named_entity', 'r_feature_politeness_==Apologizing==',
       'l_feature_politeness_==Apologizing==',
       'q_feature_politeness_==Apologizing==',
       'e_feature_politeness_==1st_person_start==',
       'e_feature_politeness_==Indirect_(btw)==',
       'e_feature_politeness_==Direct_start==',
       'e_feature_politeness_==1st_person==',
                              'l_has_adverb',
       'r_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==Deference==',
       'e_feature_politeness_==Indirect_(greeting)==',
       'l_feature_politeness_==2nd_person==',
       'e_feature_politeness_==Gratitude==',
       'e_feature_politeness_==Please_start==', 'q_question_lexicon',
       'r_has_named_entity',
                              'q_feature_politeness_==Direct_question==',
       'q_has_modal', 'r_feature_politeness_==HASHEDGE==',
       'q_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Direct_question==',
       'r_feature_politeness_==Direct_question==',
       'l_feature_politeness_==HASHEDGE==',
       'r_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==HASPOSITIVE==',
                              'q_feature_politeness_==Gratitude==',
       'q_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person_start==',
       'l_feature_politeness_==Deference==', 'r_has_cardinal_num',
       'q_feature_politeness_==1st_person_start==',
       'q_feature_politeness_==2nd_person==',
       'l_feature_politeness_==Please==',
                              'l_feature_politeness_==Factuality==', 
       'e_feature_politeness_==HASPOSITIVE==',
       'e_has_modal', 'e_question_lexicon',
                              'e_has_named_entity',
       'e_feature_politeness_==HASNEGATIVE==',
       'e_feature_politeness_==Hedges==', 'e_feature_politeness_==HASHEDGE==',
       'e_has_adjective', 'e_has_adverb']

nominal_corr_df = expanded_df[['hit_order','gold_coarse_label', 'gold_act', 'gold_intent', 
       'gold_worker', 'gold_work_time',
       'r_feature_politeness_==Factuality==',
       'r_feature_politeness_==Hedges==', 'e_feature_politeness_==Please==',
       'l_feature_politeness_==Please_start==',
       'e_feature_politeness_==SUBJUNCTIVE==', 'l_question_lexicon',
       'l_feature_politeness_==Direct_start==', 'q_has_cardinal_num',
                              'e_feature_politeness_==2nd_person==',
       'l_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person==',
       'l_feature_politeness_==HASNEGATIVE==',
       'q_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==Hedges==', 'e_has_cardinal_num', 'r_has_adverb',
       'e_feature_politeness_==Deference==',
                              'l_has_modal',
       'l_feature_politeness_==1st_person==',
       'r_feature_politeness_==SUBJUNCTIVE==',
       'e_feature_politeness_==Apologizing==',
       'q_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==Indirect_(btw)==',
       'e_feature_politeness_==Direct_question==',
       'l_feature_politeness_==1st_person_start==',
       'e_feature_politeness_==2nd_person_start==',
                              'e_feature_politeness_==Factuality==',
       'l_feature_politeness_==SUBJUNCTIVE==',
       'q_feature_politeness_==SUBJUNCTIVE==',
       'e_feature_politeness_==INDICATIVE==',
       'l_feature_politeness_==HASPOSITIVE==',
       'q_feature_politeness_==Indirect_(btw)==',
       'q_feature_politeness_==Indirect_(greeting)==',
       'r_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==HASPOSITIVE==',
                              'l_feature_politeness_==Indirect_(greeting)==', 'l_has_adjective',
       'q_has_adjective', 'e_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==HASHEDGE==',
       'l_feature_politeness_==1st_person_pl.==',
       'r_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==Please_start==',
       'l_feature_politeness_==2nd_person_start==',
                             'r_feature_politeness_==2nd_person==', 'r_has_adjective',
       'l_has_cardinal_num', 'q_feature_politeness_==Factuality==',
       'q_has_named_entity', 'r_feature_politeness_==Gratitude==',
       'r_question_lexicon', 'r_feature_politeness_==Please==',
       'r_feature_politeness_==Indirect_(greeting)==',
                             'q_feature_politeness_==Please_start==', 'q_has_adverb',
       'q_feature_politeness_==1st_person==',
       'r_feature_politeness_==Deference==', 'r_has_modal',
       'r_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Gratitude==',
       'l_feature_politeness_==Indirect_(btw)==',
       'l_feature_politeness_==Hedges==',
                              'q_feature_politeness_==Please==',
       'l_has_named_entity', 'r_feature_politeness_==Apologizing==',
       'l_feature_politeness_==Apologizing==',
       'q_feature_politeness_==Apologizing==',
       'e_feature_politeness_==1st_person_start==',
       'e_feature_politeness_==Indirect_(btw)==',
       'e_feature_politeness_==Direct_start==',
       'e_feature_politeness_==1st_person==',
                              'l_has_adverb',
       'r_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==Deference==',
       'e_feature_politeness_==Indirect_(greeting)==',
       'l_feature_politeness_==2nd_person==',
       'e_feature_politeness_==Gratitude==',
       'e_feature_politeness_==Please_start==', 'q_question_lexicon',
       'r_has_named_entity',
                              'q_feature_politeness_==Direct_question==',
       'q_has_modal', 'r_feature_politeness_==HASHEDGE==',
       'q_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Direct_question==',
       'r_feature_politeness_==Direct_question==',
       'l_feature_politeness_==HASHEDGE==',
       'r_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==HASPOSITIVE==',
                              'q_feature_politeness_==Gratitude==',
       'q_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person_start==',
       'l_feature_politeness_==Deference==', 'r_has_cardinal_num',
       'q_feature_politeness_==1st_person_start==',
       'q_feature_politeness_==2nd_person==',
       'l_feature_politeness_==Please==',
                              'l_feature_politeness_==Factuality==', 
       'e_feature_politeness_==HASPOSITIVE==',
       'e_has_modal', 'e_question_lexicon',
                              'e_has_named_entity',
       'e_feature_politeness_==HASNEGATIVE==',
       'e_feature_politeness_==Hedges==', 'e_feature_politeness_==HASHEDGE==',
       'e_has_adjective', 'e_has_adverb',  'gold_sentiment_num',
                             'gold_q_sentiment_num',
       'gold_sentiment_coarse_num', 'gold_q_sentiment_coarse_num',
       'gold_explanation_num_sents',
       'gold_explanation_len']]
nominal_corr_df.reset_index(inplace=True,drop=True)
get_correlations(nominal_corr_df, nominal_columns=nominal_columns)

In [ ]:
corr_gold_df = gold_df[['hearing_id', 'hearing_title', 'qa_index', 'q_speaker',
       'q_speaker_detail', 'r_speaker_detail', 'hit_order',
        'gold_label_set', 'gold_act_set', 
        'gold_intent_set',
        'r_text_num_sents', 'r_text_len',
        'q_text_num_sents', 'q_text_len', 'question_type',
       'q_text_last_question_len',
       'gold_sentiments_coarse_num',
       'gold_q_sentiments_coarse_num', 
        'q_speaker_last_name', 
        'gold_q_intents_num',
         'gold_sentiments_num', 'gold_q_sentiments_num',
        'q_speaker_role',
       'r_speaker_role', 'q_speaker_party', 
        'entropy_norm',
       'r_feature_politeness_==Factuality==',
       'gold_explanation_tokenized', 'r_feature_politeness_==Hedges==',
       #'e_feature_politeness_==Please==', 
       #'e_feature_politeness_==HASHEDGE==',
       'l_feature_politeness_==Please_start==',
       #'e_feature_politeness_==SUBJUNCTIVE==', 
       'l_question_lexicon',
       'l_feature_politeness_==Direct_start==', 'q_has_cardinal_num',
       #'e_feature_politeness_==2nd_person==',
       'l_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person==',
       'l_feature_politeness_==HASNEGATIVE==',
       'q_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==Hedges==', 
       #'e_has_cardinal_num', 
       'r_has_adverb',
       #'e_feature_politeness_==Deference==', 
       'l_has_modal',
       'l_feature_politeness_==1st_person==',
       'r_feature_politeness_==SUBJUNCTIVE==',
       #'e_feature_politeness_==HASPOSITIVE==', 'e_has_adverb',
       #'e_feature_politeness_==Apologizing==',
       'q_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==Indirect_(btw)==',
       #'e_feature_politeness_==Direct_question==',
       'l_feature_politeness_==1st_person_start==',
       #'e_feature_politeness_==HASNEGATIVE==',
       #'e_feature_politeness_==2nd_person_start==',
       #'e_feature_politeness_==Factuality==',
       'l_feature_politeness_==SUBJUNCTIVE==',
       'q_feature_politeness_==SUBJUNCTIVE==',
       #'e_feature_politeness_==INDICATIVE==',
       'l_feature_politeness_==HASPOSITIVE==',
       'q_feature_politeness_==Indirect_(btw)==', 
       #'e_has_adjective',
       'q_feature_politeness_==Indirect_(greeting)==',
       'r_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==HASPOSITIVE==',
       'l_feature_politeness_==Indirect_(greeting)==', 'l_has_adjective',
       'q_has_adjective', 
       #'e_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==HASHEDGE==',
       'l_feature_politeness_==1st_person_pl.==',
       'r_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==Please_start==',
       'l_feature_politeness_==2nd_person_start==',
       'r_feature_politeness_==2nd_person==', 'r_has_adjective',
       'l_has_cardinal_num', 'q_feature_politeness_==Factuality==',
       'q_has_named_entity', 'r_feature_politeness_==Gratitude==',
       'r_question_lexicon', 'r_feature_politeness_==Please==',
       'r_feature_politeness_==Indirect_(greeting)==',
       'q_feature_politeness_==Please_start==', 'q_has_adverb',
       'q_feature_politeness_==1st_person==',
       'r_feature_politeness_==Deference==', 'r_has_modal',
       'r_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Gratitude==',
       'l_feature_politeness_==Indirect_(btw)==',
       'l_feature_politeness_==Hedges==', 'q_feature_politeness_==Please==',
       'l_has_named_entity', 'r_feature_politeness_==Apologizing==',
       'l_feature_politeness_==Apologizing==',
       'q_feature_politeness_==Apologizing==',
       #'e_feature_politeness_==1st_person_start==',
       #'e_feature_politeness_==Indirect_(btw)==',
       #'e_feature_politeness_==Direct_start==',
       #'e_feature_politeness_==1st_person==',
       #'e_feature_politeness_==Hedges==', 
        'l_has_adverb',
       'r_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==Deference==',
       #'e_feature_politeness_==Indirect_(greeting)==',
       'l_feature_politeness_==2nd_person==',
       #'e_feature_politeness_==Gratitude==',
       #'e_feature_politeness_==Please_start==', 'q_question_lexicon',
       'r_has_named_entity', 'q_feature_politeness_==Direct_question==',
       #'e_has_modal', 
       'q_has_modal', 'r_feature_politeness_==HASHEDGE==',
       'q_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Direct_question==',
       'r_feature_politeness_==Direct_question==',
       'l_feature_politeness_==HASHEDGE==',
       'r_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==HASPOSITIVE==',
       'q_feature_politeness_==Gratitude==', #'e_question_lexicon',
       'q_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person_start==',
       'l_feature_politeness_==Deference==', 'r_has_cardinal_num',
       'q_feature_politeness_==1st_person_start==',
       'q_feature_politeness_==2nd_person==',
       'l_feature_politeness_==Please==',
       'l_feature_politeness_==Factuality=='#, 'e_has_named_entity'
                       ]]
get_correlations(corr_gold_df, nominal_columns=[
    'gold_sentiments_coarse_num','gold_q_sentiments_coarse_num',
    'gold_q_intents_num', 
    'gold_sentiments_num', 'gold_q_sentiments_num',
    'hearing_id', 'hearing_title', 'qa_index', 'q_speaker',
       'q_speaker_detail', 'r_speaker_detail',
       'gold_label_set', 'gold_act_set', 
       'gold_intent_set',
       'question_type',
       'q_speaker_last_name',
       'q_speaker_role',
       'r_speaker_role', 'q_speaker_party', 
       'r_feature_politeness_==Factuality==',
       'gold_explanation_tokenized', 'r_feature_politeness_==Hedges==',
       'e_feature_politeness_==Please==', 'e_feature_politeness_==HASHEDGE==',
       'l_feature_politeness_==Please_start==',
       'e_feature_politeness_==SUBJUNCTIVE==', 'l_question_lexicon',
       'l_feature_politeness_==Direct_start==', 'q_has_cardinal_num',
       'e_feature_politeness_==2nd_person==',
       'l_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person==',
       'l_feature_politeness_==HASNEGATIVE==',
       'q_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==Hedges==', 'e_has_cardinal_num', 'r_has_adverb',
       'e_feature_politeness_==Deference==', 'l_has_modal',
       'l_feature_politeness_==1st_person==',
       'r_feature_politeness_==SUBJUNCTIVE==',
       'e_feature_politeness_==HASPOSITIVE==', 'e_has_adverb',
       'e_feature_politeness_==Apologizing==',
       'q_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==Indirect_(btw)==',
       'e_feature_politeness_==Direct_question==',
       'l_feature_politeness_==1st_person_start==',
                       'e_feature_politeness_==HASNEGATIVE==',
       'e_feature_politeness_==2nd_person_start==',
       'e_feature_politeness_==Factuality==',
       'l_feature_politeness_==SUBJUNCTIVE==',
       'q_feature_politeness_==SUBJUNCTIVE==',
       'e_feature_politeness_==INDICATIVE==',
       'l_feature_politeness_==HASPOSITIVE==',
       'q_feature_politeness_==Indirect_(btw)==', 'e_has_adjective',
       'q_feature_politeness_==Indirect_(greeting)==',
       'r_feature_politeness_==2nd_person_start==',
       'q_feature_politeness_==HASPOSITIVE==',
       'l_feature_politeness_==Indirect_(greeting)==', 'l_has_adjective',
       'q_has_adjective', 'e_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==HASHEDGE==',
       'l_feature_politeness_==1st_person_pl.==',
       'r_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==Please_start==',
       'l_feature_politeness_==2nd_person_start==',
       'r_feature_politeness_==2nd_person==', 'r_has_adjective',
       'l_has_cardinal_num', 'q_feature_politeness_==Factuality==',
       'q_has_named_entity', 'r_feature_politeness_==Gratitude==',
       'r_question_lexicon', 'r_feature_politeness_==Please==',
       'r_feature_politeness_==Indirect_(greeting)==',
       'q_feature_politeness_==Please_start==', 'q_has_adverb',
       'q_feature_politeness_==1st_person==',
       'r_feature_politeness_==Deference==', 'r_has_modal',
       'r_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Gratitude==',
       'l_feature_politeness_==Indirect_(btw)==',
       'l_feature_politeness_==Hedges==', 'q_feature_politeness_==Please==',
       'l_has_named_entity', 'r_feature_politeness_==Apologizing==',
       'l_feature_politeness_==Apologizing==',
       'q_feature_politeness_==Apologizing==',
       'e_feature_politeness_==1st_person_start==',
       'e_feature_politeness_==Indirect_(btw)==',
       'e_feature_politeness_==Direct_start==',
       'e_feature_politeness_==1st_person==',
       'e_feature_politeness_==Hedges==', 'l_has_adverb',
       'r_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==Deference==',
       'e_feature_politeness_==Indirect_(greeting)==',
       'l_feature_politeness_==2nd_person==',
       'e_feature_politeness_==Gratitude==',
       'e_feature_politeness_==Please_start==', 'q_question_lexicon',
       'r_has_named_entity', 'q_feature_politeness_==Direct_question==',
       'e_has_modal', 'q_has_modal', 'r_feature_politeness_==HASHEDGE==',
       'q_feature_politeness_==Direct_start==',
       'l_feature_politeness_==Direct_question==',
       'r_feature_politeness_==Direct_question==',
       'l_feature_politeness_==HASHEDGE==',
       'r_feature_politeness_==HASNEGATIVE==',
       'r_feature_politeness_==HASPOSITIVE==',
       'q_feature_politeness_==Gratitude==', 'e_question_lexicon',
       'q_feature_politeness_==1st_person_pl.==',
       'q_feature_politeness_==INDICATIVE==',
       'r_feature_politeness_==1st_person_start==',
       'l_feature_politeness_==Deference==', 'r_has_cardinal_num',
       'q_feature_politeness_==1st_person_start==',
       'q_feature_politeness_==2nd_person==',
       'l_feature_politeness_==Please==',
       'l_feature_politeness_==Factuality=='#, 'e_has_named_entity'
                                               ])

In [ ]:
get_correlations(gold_df[['entropy_norm','q_speaker','gold_label_set','hit_order']], nominal_columns=['q_speaker','gold_label_set'])

## Q4: How does sentiment change over hearing?

In [ ]:
worker_to_color = {}
worker_colors = (plt.get_cmap('tab20b').colors)+(plt.get_cmap('tab20c').colors)+(plt.get_cmap('tab20b').colors+(plt.get_cmap('tab20c').colors))
for idx, worker in enumerate(expanded_df.sort_values(by=['gold_worker'])['gold_worker'].unique()):
    worker_to_color[worker] = worker_colors[idx]
    
def plot_sentiment_time(df, column):        
    n_hearings = len(df['hearing_id'].unique())
    n_cols=4
    n_rows=math.ceil(n_hearings / n_cols)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
    grouped = df.groupby(['hearing_id'])
    row_idx, col_idx = 0, 0
    n_figs = 0
    for name, group in grouped:
        n_figs+=1
        plot_title = name[-5:]+':'+group['hearing_title'].unique()[0][:20]
        subplot_sentiment_time(axes[row_idx, col_idx], group, column, plot_title, worker_to_color)
        # increment counters
        col_idx += 1
        if col_idx >= n_cols:
            col_idx = 0
            row_idx += 1
    fig.tight_layout()
    plt.show()
    
def subplot_sentiment_time(ax, df, column, plot_title, worker_to_color):
    qas = df.sort_values(by=['qa_index'])['qa_index'].unique()
    workers = df['gold_worker'].unique()
    for worker in workers:
        worker_qas = df[df['gold_worker']==worker].sort_values(by=['qa_index'])['qa_index'].unique()
        x = []
        y = []
        for worker_qa in worker_qas:
            x.append(np.where(qas==worker_qa)[0][0])
            y.append(df[(df['gold_worker']==worker) & (df['qa_index']==worker_qa)][column].values[0])
        #print('going to plot x', x, 'and y', y)
        ax.plot(x, y, color=worker_to_color[worker],marker='.')

    #ax.set_yticklabels(range(-3,4,1))
    ax.set_ylim(-4, 4)
    ax.set_title(plot_title)
    ax.set_ylabel('Sentiment')
    #y = [len(labels) for labels in df[column].values]
    #x = range(1,51)
    #ax.plot(x,y,marker='o', linestyle=':')

In [ ]:
def plot_sentiment_change_time(df, column):
    n_hearings = len(df['hearing_id'].unique())
    n_cols=4
    n_rows=math.ceil(n_hearings / n_cols)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12, 8))
    grouped = df.groupby(['hearing_id'])
    row_idx, col_idx = 0, 0
    n_figs = 0
    for name, group in grouped:
        n_figs+=1
        plot_title = name[-5:]+':'+group['hearing_title'].unique()[0][:20]
        #print('hearing', plot_title)
        subplot_sentiment_change_time(axes[row_idx, col_idx], group, column, plot_title, worker_to_color)
        # increment counters
        col_idx += 1
        if col_idx >= n_cols:
            col_idx = 0
            row_idx += 1
    fig.tight_layout()
    plt.show()

workerhearing_to_num_sentiment_changes = collections.defaultdict(int)
workerhearing_to_num_sentiment_flips = collections.defaultdict(int)
def subplot_sentiment_change_time(ax, df, column, plot_title, worker_to_color):
    hearing_id = df['hearing_id'].unique()[0]
    print(hearing_id)
    worker_to_prev_sentiment = {}
    hearing_workers = df['gold_worker'].unique()
    for worker in hearing_workers:
        x=[]
        y=[]
        #print('worker ', worker)
        for idx, qa in enumerate(df.sort_values(by=['qa_index'])['qa_index'].unique()):
            #print('index', idx, 'qa', qa)
            qa_workers = df[df['qa_index']==qa]['gold_worker'].unique()
            curr_sentiment = None
            new_change_count = None
            if worker in qa_workers:
                if worker+':'+hearing_id not in workerhearing_to_num_sentiment_flips:
                    workerhearing_to_num_sentiment_flips[worker+':'+hearing_id]=0
                if worker+':'+hearing_id not in workerhearing_to_num_sentiment_changes:
                    workerhearing_to_num_sentiment_changes[worker+':'+hearing_id]=0
                #print('worker annotated question')
                curr_sentiment = df[(df['gold_worker']==worker) & (df['qa_index']==qa)][column].values[0]
                #print('curr sentiment', curr_sentiment)
                if worker in worker_to_prev_sentiment:
                    #print('worker has prev sent')
                    prev_sentiment = worker_to_prev_sentiment[worker]
                    max_change_count = max((count for count in y if count is not None), default=None)
                    #print('current max change count is', max_change_count)
                    if max_change_count is None:
                        #print('setting max change count to 0 because all None in y')
                        max_change_count = 0
                    if prev_sentiment and prev_sentiment != curr_sentiment:
                        #print('found change in sentiment!')
                        if ('ositive' in prev_sentiment and 'egative' in curr_sentiment) or ('egative' in prev_sentiment and 'ositive' in curr_sentiment):
                            new_change_count = max_change_count + 1
                            workerhearing_to_num_sentiment_flips[worker+':'+hearing_id] += 1
                        else:
                            new_change_count = max_change_count + 0
                            workerhearing_to_num_sentiment_changes[worker+':'+hearing_id] += 1
                    else:
                        new_change_count = max_change_count
                    #print('new change count is now', new_change_count)
                else:
                    #print('worker has no prev sent')
                    new_change_count = 0
                worker_to_prev_sentiment[worker] = curr_sentiment
            else:
                #print('worker did not annotate question')
                new_change_count = None
            x.append(idx)
            #print('added change count value', new_change_count)
            y.append(new_change_count)
        #if '88494' in plot_title:
        #    print('going to plot x', x, 'and y', y)
        ax.plot(x, y, color=worker_to_color[worker],marker='.')
    ax.set_ylim(0, 9)
    ax.set_title(plot_title)
    ax.set_ylabel(r'$\Delta$' ' sentiment')
 

In [ ]:
plot_sentiment_change_time(expanded_df, 'gold_sentiment_coarse')

In [ ]:
all_workers = gold_df[gold_df['hearing_id']=='CHRG-116hhrg36001']['gold_workers'].values
all_workers=[list(worker_group) for worker_group in all_workers]
common_workers = collections.Counter(list(chain.from_iterable(all_workers))).most_common(5)

In [ ]:
worker1=expanded_df[(expanded_df['hearing_id']=='CHRG-116hhrg36001') & (expanded_df['gold_worker']==common_workers[0][0])].sort_values(by='qa_index')[['qa_index','gold_sentiment_coarse_num']].values
worker2=expanded_df[(expanded_df['hearing_id']=='CHRG-116hhrg36001') & (expanded_df['gold_worker']==common_workers[1][0])].sort_values(by='qa_index')[['qa_index','gold_sentiment_coarse_num']].values
worker3=expanded_df[(expanded_df['hearing_id']=='CHRG-116hhrg36001') & (expanded_df['gold_worker']==common_workers[2][0])].sort_values(by='qa_index')[['qa_index','gold_sentiment_coarse_num']].values
worker4=expanded_df[(expanded_df['hearing_id']=='CHRG-116hhrg36001') & (expanded_df['gold_worker']==common_workers[3][0])].sort_values(by='qa_index')[['qa_index','gold_sentiment_coarse_num']].values

In [ ]:
index_to_num = dict(zip(gold_df[gold_df['hearing_id']=='CHRG-116hhrg36001'].sort_values(by=['qa_index'])['qa_index'].values, range(len(gold_df[gold_df['hearing_id']=='CHRG-116hhrg36001'].sort_values(by=['qa_index'])['qa_index'].values))))

In [ ]:
worker1_qnums = [index_to_num[worker_info] for worker_info in worker1[:,0]]
worker2_qnums = [index_to_num[worker_info] for worker_info in worker2[:,0]]
worker3_qnums = [index_to_num[worker_info] for worker_info in worker3[:,0]]
worker4_qnums = [index_to_num[worker_info] for worker_info in worker4[:,0]]

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.plot(worker1_qnums, worker1[:,1],  zorder=1, lw=1)
plt.scatter(worker1_qnums, worker1[:,1],zorder=2)
plt.title('Worker 1')
plt.ylabel('Coarse-grained Sentiment')
plt.xlabel('Question Number')
plt.ylim(-1.2,1.2)
plt.yticks(np.arange(-1, 2, 1))

In [ ]:
plt.plot(worker2_qnums, worker2[:,1],  zorder=1, lw=1)
plt.scatter(worker2_qnums, worker2[:,1],zorder=2)
plt.title('Worker 2')
plt.ylabel('Coarse-grained Sentiment')
plt.xlabel('Question Number')
plt.ylim(-1.2,1.2)
plt.yticks(np.arange(-1, 2, 1))

In [ ]:
plt.plot(worker3_qnums, worker3[:,1],  zorder=1, lw=1)
plt.scatter(worker3_qnums, worker3[:,1],zorder=2)
plt.title('Worker 3')
plt.ylabel('Coarse-grained Sentiment')
plt.xlabel('Question Number')
plt.ylim(-1.2,1.2)
plt.yticks(np.arange(-1, 2, 1))

In [ ]:
plt.plot(worker4_qnums, worker4[:,1],  zorder=1, lw=1)
plt.scatter(worker4_qnums, worker4[:,1],zorder=2)
plt.title('Worker 4')
plt.ylabel('Coarse-grained Sentiment')
plt.xlabel('Question Number')
plt.ylim(-1.2,1.2)
plt.yticks(np.arange(-1, 2, 1))

In [ ]:
worker_to_num_flips = collections.defaultdict(int)
hearing_to_num_flips = collections.defaultdict(int)
for workerhearing, num_flips in workerhearing_to_num_sentiment_flips.items():
    parts = workerhearing.split(':')
    worker = parts[0]
    hearing = parts[1]
    worker_to_num_flips[worker] += num_flips
    hearing_to_num_flips[hearing] += num_flips
(np.array(list(worker_to_num_flips.values()))>0).sum()

In [ ]:
worker_to_num_changes = collections.defaultdict(int)
hearing_to_num_changes = collections.defaultdict(int)
for workerhearing, num_changes in workerhearing_to_num_sentiment_changes.items():
    parts = workerhearing.split(':')
    worker = parts[0]
    hearing = parts[1]
    worker_to_num_changes[worker] += num_changes
    hearing_to_num_changes[hearing] += num_changes
(np.array(list(worker_to_num_changes.values()))>0).sum()

In [ ]:
#{k: v for k, v in sorted(hearing_to_num_flips.items(), key=lambda item: item[1])}
df_flips = pd.DataFrame(hearing_to_num_flips.items(),columns=['hearing_id','num_flips'])
df_changes = pd.DataFrame(hearing_to_num_changes.items(),columns=['hearing_id','num_changes'])

In [ ]:
df_flips.sort_values(by=['num_flips'])

In [ ]:
df_changes.sort_values(by=['num_changes'])

In [ ]:
direct_answers_series = expanded_df[expanded_df['gold_coarse_label']=='answer'].groupby(['hearing_id'])['qa_index'].count()
flips_answers_df = pd.merge(df_flips, direct_answers_series, left_on='hearing_id', right_index=True)
changes_answers_df = pd.merge(df_changes, direct_answers_series, left_on='hearing_id', right_index=True)

In [ ]:
flips_answers = flips_answers_df[['num_flips','qa_index']].values
changes_answers = changes_answers_df[['num_changes','qa_index']].values

In [ ]:
num_flips, num_answers = list(zip(*flips_answers))
from scipy.stats import pearsonr
pearsonr(num_flips, num_answers)

In [ ]:
num_changes, num_answers = list(zip(*changes_answers))
from scipy.stats import pearsonr
pearsonr(num_changes, num_answers)

In [ ]:
(expanded_df.groupby(['gold_worker'])['gold_hit_id'].count()>5).sum()

In [ ]:
worker_to_hit_ids = expanded_df.groupby(['gold_worker'])['gold_hit_id'].unique().to_dict()
worker_to_num_hits = {}
for worker, hits in worker_to_hit_ids.items():
    worker_to_num_hits[worker] = len(hits)


In [ ]:
worker_to_sent_change_ratio={}
for worker, num_changes in worker_to_num_changes.items():
    worker_to_sent_change_ratio[worker] = num_changes/worker_to_num_hits[worker]

In [ ]:
worker_to_sent_change_ratio

In [ ]:
worker_to_sent_flip_ratio={}
for worker, num_flips in worker_to_num_flips.items():
    worker_to_sent_flip_ratio[worker] = num_flips/worker_to_num_hits[worker]
worker_to_sent_flip_ratio

In [ ]:
plot_sentiment_time(expanded_df, 'gold_sentiment_coarse_num')

In [ ]:
plot_sentiment_time(expanded_df, 'gold_sentiment_num')

In [ ]:
plot_sentiment_time(expanded_df, 'gold_q_sentiment_coarse_num')

In [ ]:
plot_sentiment_time(expanded_df, 'gold_q_sentiment_num')

## Q5: Are there words indicative of labels?

In [ ]:
class PmiLmiCalculator(object):
    def __init__(self):
        self.label_to_counts = {}
        self.word_to_index = {}
        self.all_counts = {}
        self.vectorizer = None
        
    def get_highest_pmi_lmi(self, df, text_column, label_column, label, save_file, n_gram, min_freq, top_k=20):
        print('Calculating PMI and LMI for label', label, 'with ngram', n_gram, 'min freq', min_freq)
        self.set_all_counts_and_index(df, text_column, n_gram, min_freq)
        self.set_label_counts(df, text_column, label_column, label)
        matrix_pmi, matrix_lmi = self.calculate_pmi_lmi(self.label_to_counts[label])

        sorted_pmi = np.array(np.argsort(matrix_pmi)).reshape(-1, )  # from lowest to highest
        sorted_pmi = sorted_pmi[::-1]  # reverse, from highest to lowest
        sorted_pmi_words = [self.lookup_value(self.word_to_index, index) for index in sorted_pmi]
        sorted_pmi_scores = np.array(matrix_pmi[0,sorted_pmi]).reshape(-1, )*10**4
        sorted_pmi_freqs = np.array(self.label_to_counts[label][:,sorted_pmi].sum(axis=0)).reshape(-1,)
        sorted_pmi_tuples = list(zip(sorted_pmi_scores, sorted_pmi_words, sorted_pmi_freqs))
        with open(save_file+'_pmi.csv','w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow(['pmi_score','word', 'frequency'])
            for row in sorted_pmi_tuples:
                csv_out.writerow(row)
        print('Saved PMI scores to ', save_file+'_pmi.csv')

        top_k_pmi_words = sorted_pmi_tuples[:top_k]

        sorted_lmi = np.array(np.argsort(matrix_lmi)).reshape(-1, )  # from lowest to highest
        sorted_lmi = sorted_lmi[::-1]  # reverse, from highest to lowest
        sorted_lmi_words = [self.lookup_value(self.word_to_index, index) for index in sorted_lmi]
        sorted_lmi_scores = np.array(matrix_lmi[0,sorted_lmi]).reshape(-1, )*10**4
        sorted_lmi_freqs = np.array(self.label_to_counts[label][:,sorted_lmi].sum(axis=0)).reshape(-1,)
        sorted_lmi_tuples = list(zip(sorted_lmi_scores, sorted_lmi_words, sorted_lmi_freqs))
        # move nan's to the end of the list
        sorted_lmi_tuples = sorted(sorted_lmi_tuples, key=lambda x: np.isnan(x[0])) 

        with open(save_file+'_lmi.csv','w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow(['lmi_score','word', 'frequency'])
            for row in sorted_lmi_tuples:
                csv_out.writerow(row)
        print('Saved LMI scores to ', save_file+'_lmi.csv')
        top_k_lmi_words = sorted_lmi_tuples[:top_k]

        return top_k_pmi_words, top_k_lmi_words

    def lookup_value(self, _dict, _value):
        return next(key for key, value in _dict.items() if value == _value)

    def set_all_counts_and_index(self, df, text_column, n_gram, min_freq):
        print('inside set')
        if not self.word_to_index:
            print('Creating counter for words...')
            vocab_vectorizer = CountVectorizer(ngram_range=(n_gram, n_gram), tokenizer=tokenize_text, 
                                               min_df=min_freq)#token_pattern='*', 
            vocab_vectorizer.fit_transform(df[text_column].values)
            fixed_vocab = list(vocab_vectorizer.vocabulary_.keys())
            self.vectorizer = CountVectorizer(ngram_range=(n_gram, n_gram), tokenizer=tokenize_text, 
                                         vocabulary=fixed_vocab, #token_pattern='*', 
                                         lowercase=False, min_df=min_freq)
            self.all_counts = self.vectorizer.fit_transform(df[text_column].values)
            self.word_to_index = self.vectorizer.vocabulary_
            print('Created counter for all words with vocab of size', len(self.word_to_index.keys()))

    def set_label_counts(self, df, text_column, label_column, label):
        if label not in self.label_to_counts:
            print('Getting word counts for label...')
            label_counts = self.vectorizer.fit_transform(df[df[label_column]==label][text_column].values) 
            self.label_to_counts[label] = label_counts

    # p (w,l) = count(w,l) / |D|
    # p(l) = count(l) / |D|
    # p (l|w) = count(w,l) / count(w)
    # PMI = log ( p(l|w) / p(l) )
    # LMI = p(w, l) · PMI
    def calculate_pmi_lmi(self, label_counts):
        count_word_label = label_counts.sum(axis=0)
        count_label = np.sum(label_counts)
        count_word =self.all_counts.sum(axis=0)
        D = np.sum(self.all_counts)

        prob_word_label = count_word_label / D
        prob_label_word = count_word_label / count_word
        prob_label = count_label / D

        # divide prob label given word by prob label
        pmi_matrix = prob_label_word / prob_label

        # take log -- this will generate a divide by zero warning because we are taking log of 0
        pmi_matrix = np.log(pmi_matrix) 
        # pmi_matrix = np.maximum(pmi_matrix, 0) # log(0)=-inf so sub this with 0

        # multiply by prob word and label
        lmi_matrix = np.multiply(prob_word_label, pmi_matrix)
        print("Calculated lmi matrix: ", lmi_matrix.shape)
        return pmi_matrix, lmi_matrix

### Cue words in response

In [ ]:
ngrams = [1, 2, 3]
minfreqs = [5]
label_column = 'gold_label_set'
text_column = 'r_text'
df = gold_df
for ngram in ngrams:
    for minfreq in minfreqs:
        pmiLmiCalc = PmiLmiCalculator()
        for label_set in df[label_column].unique():
            label_set_name = '+'.join(label_set)
            save_file = os.path.join('../../data/analyses','lmi_pmi_response','label_sets',label_set_name+'_ngram'+str(ngram)+'_minfreq'+str(minfreq))
            pmiLmiCalc.get_highest_pmi_lmi(df, text_column, label_column, label_set, save_file, n_gram=ngram, min_freq=minfreq, top_k=50)

In [ ]:
ngrams = [1, 2, 3]
minfreqs = [5]
label_column = 'gold_coarse_label'
text_column = 'r_text'
df = expanded_ling_df
for ngram in ngrams:
    for minfreq in minfreqs:
        pmiLmiCalc = PmiLmiCalculator()
        for label in df[label_column].unique():
            save_file = os.path.join('../../data/analyses','lmi_pmi_response','labels',label+'_ngram'+str(ngram)+'_minfreq'+str(minfreq))
            pmiLmiCalc.get_highest_pmi_lmi(df, text_column, label_column, label, save_file, n_gram=ngram, min_freq=minfreq, top_k=50)

### Cue words in question

In [ ]:
ngrams = [1, 2, 3]
minfreqs = [5]
label_column = 'gold_coarse_label'
text_column = 'q_text'
df = expanded_ling_df
for ngram in ngrams:
    for minfreq in minfreqs:
        pmiLmiCalc = PmiLmiCalculator()
        for label in ['cant-answer-sincere']:#df[label_column].unique():
            save_file = os.path.join('../../data/analyses','lmi_pmi_question','labels',label+'_ngram'+str(ngram)+'_minfreq'+str(minfreq))
            pmiLmiCalc.get_highest_pmi_lmi(df, text_column, label_column, label, save_file, n_gram=ngram, min_freq=minfreq, top_k=50)

### Cue words in explanation

In [ ]:
ngrams = [1, 2, 3]
minfreqs = [5]
label_column = 'gold_coarse_label'
text_column = 'gold_explanation'
df = expanded_ling_df
for ngram in ngrams:
    for minfreq in minfreqs:
        pmiLmiCalc = PmiLmiCalculator()
        for label in df[label_column].unique():
            save_file = os.path.join('../../data/analyses','lmi_pmi_explanation','labels',label+'_ngram'+str(ngram)+'_minfreq'+str(minfreq))
            pmiLmiCalc.get_highest_pmi_lmi(df, text_column, label_column, label, save_file, n_gram=ngram, min_freq=minfreq, top_k=50)

### Get corpus statistics

In [ ]:
expanded_df[['r_text_len','q_text_len','gold_explanation_len']].describe()

In [ ]:
expanded_df[expanded_df['r_text_len']==1][['qa_index','r_text','gold_coarse_label']]

In [ ]:
expanded_df.groupby(['gold_coarse_label'])[['r_text_len','q_text_len','gold_explanation_len']].describe()

In [ ]:
expanded_df[['r_text_num_sents','q_text_num_sents','gold_explanation_num_sents']].describe()

In [ ]:
expanded_df.groupby(['gold_coarse_label'])[['r_text_num_sents','q_text_num_sents','gold_explanation_num_sents']].describe()

### Incongruous sentiment and label

In [ ]:
gold_df['gold_sentiments_string']=gold_df['gold_sentiments'].astype(str)

In [ ]:
gold_df[~(gold_df['gold_label_set']==tuple(['answer'])) & (gold_df['entropy']==0) & (gold_df['gold_sentiments_string'].str.contains('ositive')) & (gold_df['gold_sentiments_string'].str.contains('egative'))]

In [ ]:
len(gold_df[(gold_df['gold_label_set']==tuple(['answer'])) & (gold_df['entropy']==0) & (gold_df['gold_sentiments_string'].str.contains('ositive')) & (gold_df['gold_sentiments_string'].str.contains('egative'))])

In [ ]:
gold_df[gold_df['gold_label_set']==tuple(['cant-answer-lying'])]

In [ ]:
entropy_values = gold_df.sort_values(by=['entropy'])['entropy'].unique()

In [ ]:
entropy_dfs = []
entropy_mean_dfs = []
for entropy_value in entropy_values:
    entropy_df = gold_df[gold_df['entropy']==entropy_value]
    entropy_mean_dfs.append(entropy_df.mean())
    entropy_dfs.append(entropy_df)

In [ ]:
# single label: 100 vs 0
entropy_dfs[0][['qa_index','gold_label_set','gold_coarse_labels']]

In [ ]:
# 2 labels: 71% vs. 29% (5 vs. 2)
entropy_dfs[1][['qa_index','gold_label_set','gold_coarse_labels']]

In [ ]:
# 2 labels: 60% vs. 40% (3 vs. 2)
entropy_dfs[3][['qa_index','gold_label_set','gold_coarse_labels']]

In [ ]:
# 2 labels: 57% vs. 43% (4 vs. 3)
entropy_dfs[4][['qa_index','gold_label_set','gold_coarse_labels']]

In [ ]:
# 2 labels: 50% vs. 50% (3 vs. 3)
entropy_dfs[5][['qa_index','gold_label_set','gold_coarse_labels']]

In [ ]:
# 3 labels: 43% vs. 29% vs. 29% (3 vs. 2 vs. 2)
entropy_dfs[6][['qa_index','gold_label_set','gold_coarse_labels']]

In [ ]:
# 3 labels: 33% vs. 33% vs. 33% (2 vs. 2 vs. 2)
display(entropy_dfs[7][['qa_index','gold_label_set','gold_coarse_labels']])

In [ ]:
gold_df['entropy'].hist(by=gold_df['q_speaker'], figsize=(8,10))

In [ ]:
gold_df['q_speaker_fixed'].unique()

In [ ]:
def fix_speaker(speaker):
    speaker = speaker.rstrip('.')
    speaker_parts = speaker.split(' ')
    speaker_parts = [part.strip(' ') for part in speaker_parts if len(part.strip(' '))>0]
    return ' '.join(speaker_parts)

In [ ]:
gold_df['q_speaker_fixed'] = gold_df.apply(lambda x: fix_speaker(x['q_speaker']), axis=1)

In [ ]:
len(gold_df['q_speaker_fixed'].unique())

In [ ]:
' '.join(parts)

In [ ]:
get_correlations(gold_df[['entropy_norm','q_speaker_fixed','gold_label_set','hit_order']], nominal_columns=['q_speaker_fixed','gold_label_set'])